In [1]:
# Set up libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
from configparser import ConfigParser
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 


Bad key "text.kerning_factor" on line 4 in
C:\Users\HOME\Anaconda3\envs\SEGP\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Configure the connection
config = ConfigParser()
config.read("config.ini")
options = config["DataBase"]

engine = create_engine("mysql://" + options["user"] + ":" + options["passwd"] + "@" + options["host"] + "/" + options["database"])
engine.connect()

In [3]:
station = "SELECT number FROM dublinbikes.static_data"
stat_df = pd.read_sql(station,engine)

In [15]:
for station in stat_df.number:
    # Set up the SQL command and turn into dataframe
    station_data = "SELECT * FROM dublinbikes.dynamic_data WHERE number = {}".format(station)
    sql_df = pd.read_sql(station_data,engine)
    ## cleaning data
    # converting 'last_update' enties from 13 digits to the required 10
    for i in range(sql_df.shape[0]):
        convert = str(sql_df['last_update'][i])
        if len(convert) > 10:
            sql_df.at[i,'last_update'] = int(convert[:10])
    # removing duplicates
    sql_df = sql_df.drop_duplicates()
    sql_df = sql_df.reset_index(drop=True)
    # convert update to datetime
    sql_df['last_update'] = sql_df['last_update'].apply(lambda x: datetime.fromtimestamp(x))
    # adding day of the week column
    sql_df.insert(2, 'day_of_week', 0)
    sql_df['day_of_week'] = sql_df['last_update'].apply(lambda x: x.dayofweek )
    # remove lines of repeated data
    r_day, r_stand, r_bike = 0,0,0
    clean_sql_df = pd.DataFrame()
    for i in range(sql_df.shape[0]):
        c_day = sql_df['day_of_week'][i]
        c_stand = sql_df['available_bike_stands'][i]
        c_bike = sql_df['available_bikes'][i]
        if r_day == c_day and r_stand == c_stand and r_bike == c_bike:
            continue
        else:
            r_day, r_stand, r_bike = c_day, c_stand, c_bike
            clean_sql_df = clean_sql_df.append([sql_df.iloc[i]])
    clean_sql_df = clean_sql_df.reset_index(drop=True)
    ## preparing data
    usage_data = pd.DataFrame(columns=['day_of_week', 'usage'])
    r_day,r_count = 0,0
    for i in range(clean_sql_df.shape[0]):
        c_day = clean_sql_df['day_of_week'][i]
        date = clean_sql_df['last_update'][i]
        if date.month == 3 and date.day > 15:
            usage_data.loc[i] = [r_day] + [r_count]
            break
        if r_day != c_day and r_count == 0:
            r_day = c_day
            r_count += 1
        elif r_day != c_day and r_count > 0:
            usage_data.loc[i] = [r_day] + [r_count]
            r_day = c_day
            r_count = 1
        elif r_day == c_day:
            r_count += 1
        else:
            print("Something Odd in the Data {} {} {}".format(r_day,r_count,c_day))
    usage_data = usage_data.reset_index(drop=True)
    usage_data.to_csv ('stat_{}_predict.csv'.format(station), index = False, header=True)
    # splinting for individual predictions
    df_0 = pd.DataFrame()
    df_1 = pd.DataFrame()
    df_2 = pd.DataFrame()
    df_3 = pd.DataFrame()
    df_4 = pd.DataFrame()
    df_5 = pd.DataFrame()
    df_6 = pd.DataFrame()
    for i in range(usage_data.shape[0]):
        if usage_data['day_of_week'][i] == 0:
            df_0 = df_0.append([usage_data.iloc[i]])
        if usage_data['day_of_week'][i] == 1:
            df_1 = df_1.append([usage_data.iloc[i]])
        if usage_data['day_of_week'][i] == 2:
            df_2 = df_2.append([usage_data.iloc[i]])
        if usage_data['day_of_week'][i] == 3:
            df_3 = df_3.append([usage_data.iloc[i]])
        if usage_data['day_of_week'][i] == 4:
            df_4 = df_4.append([usage_data.iloc[i]])
        if usage_data['day_of_week'][i] == 5:
            df_5 = df_5.append([usage_data.iloc[i]])
        if usage_data['day_of_week'][i] == 6:
            df_6 = df_6.append([usage_data.iloc[i]])
    cont_features = ['day_of_week']
    X0 = df_0[cont_features]
    y0 = df_0.usage
    linreg0 = LinearRegression().fit(X0, y0)

    X1 = df_1[cont_features]
    y1 = df_1.usage
    linreg1 = LinearRegression().fit(X1, y1)

    X2 = df_2[cont_features]
    y2 = df_2.usage
    linreg2 = LinearRegression().fit(X2, y2)

    X3 = df_3[cont_features]
    y3 = df_3.usage
    linreg3 = LinearRegression().fit(X3, y3)

    X4 = df_4[cont_features]
    y4 = df_4.usage
    linreg4 = LinearRegression().fit(X4, y4)

    X5 = df_5[cont_features]
    y5 = df_5.usage
    linreg5 = LinearRegression().fit(X5, y5)

    X6 = df_6[cont_features]
    y6 = df_6.usage
    linreg6 = LinearRegression().fit(X6, y6)

    ydata = [linreg0.predict(X0.head(1))[0],linreg1.predict(X1.head(1))[0],linreg2.predict(X2.head(1))[0],linreg3.predict(X3.head(1))[0],linreg4.predict(X4.head(1))[0],linreg5.predict(X5.head(1))[0],linreg6.predict(X6.head(1))[0]]

    sql = "INSERT INTO dublinbikes.station_week_prediction (number, mon, tue, wed, thur, fri, sat, sun) VALUES ({}, {}, {}, {}, {}, {}, {}, {});".format(station,ydata[0],ydata[1],ydata[2],ydata[3],ydata[4],ydata[5],ydata[6])
    sql2 = "UPDATE dublinbikes.station_week_prediction SET mon = {}, tue = {}, wed = {}, thur = {}, fri = {}, sat = {}, sun = {} WHERE (number = {});".format(ydata[0],ydata[1],ydata[2],ydata[3],ydata[4],ydata[5],ydata[6],station)
    try:
        engine.execute(sql)
    except:
        engine.execute(sql2)
